# Perspectives in Computational Research: Homework 02

In [1]:
#Libraries
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras import losses
from keras import metrics

Using TensorFlow backend.


### Load Data

In [2]:
# Train Data
train_data = pd.read_csv('data/congress_train.csv', encoding = "ISO-8859-1")
train_title = train_data['Title'].tolist()
train_major = train_data['Major']

# Validata Data
val_data = pd.read_csv('data/congress_val.csv', encoding = "ISO-8859-1")
val_title = val_data['Title'].tolist()
val_major = val_data['Major']

# Test Data
test_data = pd.read_csv('data/congress_test.csv', encoding = "ISO-8859-1")
test_title = test_data['Title'].tolist()
test_major = test_data['Major']

### Tokenize

In [3]:
token = Tokenizer(num_words=10000) #class restricting to 10000 words

In [4]:
# Train Data
token.fit_on_texts(train_title)
train_sequence = token.texts_to_sequences(train_title)
train_x = pad_sequences(train_sequence, maxlen=100)
train_y = to_categorical(train_major)

In [5]:
# validate Data
token.fit_on_texts(val_title)
val_sequence = token.texts_to_sequences(val_title)
val_x = pad_sequences(val_sequence, maxlen=100)
val_y = to_categorical(val_major)

In [6]:
# validate Data
token.fit_on_texts(test_title)
test_sequence = token.texts_to_sequences(test_title)
test_x = pad_sequences(test_sequence, maxlen=100)
test_y = to_categorical(test_major)

### Embedding
Reference: https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [2]:
#Parameters for Task Specific Embedding
vocab_size = 10000
pad_len = 100
em_space = 64

### Basic Feed Forward Network

In [ ]:
# define the model
basic_model = Sequential()
basic_model.add(Embedding(vocab_size, em_space, input_length=pad_length))
basic_model.add(Flatten())
basic_model.add(Dense(32, activation='softmax'))

# compile the model
basic_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

#Fit and Store Model Val
basic_metrics = basic_model.fit(train_x, train_y, epochs= 50, batch_size= 512, validation_data = (val_x, val_y))

In [ ]:
#Extract Accuracy and Loss from keras callback history
basic_metrics = basic_metrics.history
val_accuracy = basic_metrics['val_acc']
val_loss = basic_metrics['val_loss']
epochs = numpy.arange(1, len(val_accuracy)+1)

#Plot Accuracy over Epochs
plt.clf()
plt.plot(epochs, val_accuracy)
plt.margins(0)
plt.title("Validation Accuracy over Range of Epochs")
plt.xlabel('Epochs')
plt.ylabel("Validation Accuracy")
plt.show()

#Plot Loss over Epochs
plt.clf()
plt.plot(epochs, val_loss)
plt.margins(0)
plt.title("Validation Loss over Range of Epochs")
plt.xlabel('Epochs')
plt.ylabel("Validation Loss")
plt.show()